In [16]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
# from cuml import SVC

In [17]:
data_path = "./Data.csv"
df = pd.read_csv(data_path)

# df = df[['networkExp', 'bedwars_level', 'Experience', 'beds_broken_bedwars', 'beds_lost_bedwars', 'coins', 'deaths_bedwars', 'diamond_resources_collected_bedwars', 'emerald_resources_collected_bedwars', 'final_deaths_bedwars', 'final_kills_bedwars', 'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars', 'losses_bedwars', 'void_final_deaths_bedwars',  'wins_bedwars','fkdr','wlr','bblr','fk_lev','bb_lev','kill_lev', "Cheat"]]
df = df[['karma', 'bedwars_level',
        # 'bedwars_loot_box',
        # 'all_timeBEDWARS__defensive', 'BEDWARS__offensive',
        # 'BEDWARS__support', 'Bedwars_openedChests',
        # 'beds_broken_bedwars', 'beds_lost_bedwars',
        # 'deaths_bedwars', 'diamond_resources_collected_bedwars',
        # 'emerald_resources_collected_bedwars', 'fall_deaths_bedwars',
        # 'final_deaths_bedwars', 'final_kills_bedwars',
        # 'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars',
        # 'losses_bedwars', 'void_deaths_bedwars', 'void_final_deaths_bedwars',
        'void_final_kills_bedwars', 'void_kills_bedwars', 'wins_bedwars', "Cheat"
        ]]

In [18]:
# Cheatカラムを削除して、各レコードの合計を計算する
row_sums = df.drop('Cheat', axis=1).sum(axis=1)

# 分母が0になる可能性がある行のインデックスを取得
zero_division_indices = row_sums[row_sums == 0].index

# 10,000をその行の合計値で割った値を計算して保持する
divisors = pd.Series(0, index=df.index)  # 全ての行を0で初期化
divisors[zero_division_indices] = 0  # 分母が0になる行のみ0に設定
divisors[~divisors.index.isin(zero_division_indices)] = 10000 / row_sums[~row_sums.index.isin(zero_division_indices)]

# それぞれの行に対して計算した値を掛ける
df_scaled = df.drop('Cheat', axis=1).mul(divisors, axis=0)

# データフレームにCheatカラムを追加する
df_scaled['Cheat'] = df['Cheat']
df_scaled

# 新しいデータベースとして保存する
# df_scaled.to_csv('ScaledData.csv', index=False)

,karma,bedwars_level,void_final_kills_bedwars,void_kills_bedwars,wins_bedwars,Cheat
0,9460.116732,16.212711,175.097276,322.632944,25.940337,0
1,1319.648094,410.557185,2199.413490,5718.475073,351.906158,0
2,9979.133185,0.907253,4.017834,13.219970,2.721759,0
3,9589.041096,21.074816,158.061117,168.598525,63.224447,0
4,0.000000,596.026490,761.589404,7814.569536,827.814570,0
...,...,...,...,...,...,...
4673,6995.478833,2.055076,524.044390,1944.101932,534.319770,1
4674,449.640288,17.985612,683.453237,6420.863309,2428.057554,1
4675,3773.584906,15.723270,707.547170,4261.006289,1242.138365,1
4676,2807.646356,11.947431,1206.690562,4086.021505,1887.694146,1


In [19]:
# df = pd.get_dummies(df)
# print("dummies")

X = df_scaled.drop('Cheat', axis=1)
y = df_scaled['Cheat']
df_scaled

# データの標準化
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)


# 不足しているデータのインデックスを特定
underrepresented_indices = df_scaled.index

missing_indices = [index for index in underrepresented_indices if index not in X_train.index]
print("Missing indices:", missing_indices)

# Filter out the missing indices
underrepresented_indices = [index for index in underrepresented_indices if index in X_train.index]
X_underrepresented = X_train.loc[underrepresented_indices]
y_underrepresented = y_train.loc[underrepresented_indices]


# 不足しているデータだけを抽出
X_underrepresented = X_train.loc[underrepresented_indices]
y_underrepresented = y_train.loc[underrepresented_indices]

# SMOTEでオーバーサンプリングを実行
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_underrepresented, y_underrepresented)

# オーバーサンプリング後のデータを元のデータに結合
X_train_resampled = pd.concat([X_train, X_resampled])
y_train_resampled = pd.concat([y_train, y_resampled])

# モデルの作成と訓練（オーバーサンプリング後のデータを使用）
model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight={0: 1, 1:10})
model.fit(X_train_resampled, y_train_resampled)



# テストデータを使って予測を行う
y_pred = model.predict(X_test)


# # モデルの作成
# model = RandomForestClassifier(n_estimators=200, random_state=42)
# model.fit(X_train, y_train)


# # モデルの訓練
# model.fit(X_train, y_train)

# # テストデータを使って予測を行う
# y_pred = model.predict(X_test)

# 正解率を計算する
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# 適合率を計算する
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# 再現率を計算する
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1スコアを計算する
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')

# 混同行列を表示する
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# モデルと標準化器の保存
# joblib.dump(model, './models/Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')


Missing indices: [39, 42, 45, 49, 57, 72, 98, 104, 113, 117, 124, 130, 134, 138, 148, 154, 159, 162, 170, 173, 189, 192, 214, 218, 220, 223, 224, 246, 269, 271, 291, 294, 302, 304, 308, 348, 379, 383, 394, 398, 402, 405, 410, 415, 453, 472, 481, 483, 489, 491, 496, 500, 502, 514, 517, 521, 526, 530, 531, 538, 541, 543, 545, 556, 562, 567, 578, 582, 584, 599, 634, 636, 639, 651, 668, 675, 684, 716, 727, 731, 776, 780, 788, 812, 825, 829, 847, 878, 883, 896, 918, 927, 933, 938, 962, 965, 980, 991, 1001, 1003, 1011, 1014, 1042, 1051, 1065, 1066, 1077, 1085, 1091, 1098, 1114, 1118, 1161, 1170, 1185, 1196, 1202, 1220, 1223, 1232, 1261, 1267, 1271, 1275, 1298, 1319, 1326, 1330, 1338, 1351, 1357, 1362, 1366, 1376, 1377, 1382, 1385, 1389, 1410, 1412, 1423, 1450, 1452, 1470, 1475, 1487, 1490, 1509, 1511, 1513, 1515, 1522, 1525, 1530, 1531, 1556, 1563, 1564, 1567, 1571, 1586, 1620, 1621, 1632, 1657, 1662, 1675, 1680, 1698, 1705, 1708, 1710, 1742, 1759, 1761, 1767, 1778, 1793, 1828, 1830, 1831, 1

In [20]:
joblib.dump(model, '../Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')

['../Cheater.pkl']

# snipe垢の条件

fkdr, wlr, bblrは低い

star n.levは30以下

sniper%が高い　（ターゲットが見つかるまで何回もｒｑするから）

fk/lev bb/lev kill/lev が高い　fk/levは7.5 bb/levは6.5 kill/levは20以上で高い

# cheatersデータ - snipe垢 = not sniper cheaters
snipe垢の条件外のcheaterを予測する。

# cubelifyのsniper%
cubelifyのsniper%を取得して、学習データとする